In [1]:
%load_ext autoreload
%autoreload 2
import os
import sys
import numpy as np
import pandas as pd
import xarray as xr
from os.path import join as pjoin
from tqdm import tqdm
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pingouin as pg
from scipy.stats import pearsonr, zscore, spearmanr

sys.path.append("../..")
import circletrack_behavior as ctb
import circletrack_neural as ctn
import plotting_functions as pf
import place_cells as pc 

/home/austinbaggetta/.conda/envs/calcium/lib/python3.10/site-packages/outdated/utils.py:14: OutdatedCacheFailedWarning: Failed to use cache while checking for outdated package.
Set the environment variable OUTDATED_RAISE_EXCEPTION=1 for a full traceback.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(
/home/austinbaggetta/.conda/envs/calcium/lib/python3.10/site-packages/outdated/utils.py:14: OutdatedPackageWarning: The package pingouin is out of date. Your version is 0.5.2, the latest is 0.5.3.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


In [26]:
## Set parameters
dataset = {}
result_dict = {'mouse': [], 'session': [], 'rotation': [], 'r': [], 'p': []}
dpath = os.path.abspath('../../../MultiCon_AfterHours/MultiCon_EEG1/output/aligned_minian')
save_path = os.path.abspath('../../../MultiCon_AfterHours/MultiCon_EEG1/output/rotated_placefields/mc_EEG1_01/')
feather_name = 'A1_A5_placecells_pearson.feat'
mouse_dict = {'mc_EEG1_01': ['A1', 'A5']} ## set which sessions you will be comparing
degree_increment = 180 ## step for x,y position rotation
forward = False ## whether or not to include only forward (correct) trials
place_cells = True ## whether or not to include only place cells
neuron_by_neuron = True ## whether to get the correlation between place fields
ref_sess = 'A1' ## reference session
alpha = 0.05
nbins = 20

for mouse in mouse_dict.keys():
    for session in mouse_dict[mouse]:
        file_name = '_'.join([mouse, session])
        da = xr.open_dataset(pjoin(dpath, f'{mouse}/{file_name}.nc'))['S'] ## select S matrix
        dataset[session] = da

    reference_session = dataset[ref_sess] ## set reference session
    if forward:
        forward_trials, reverse_trials = ctb.forward_reverse_trials(reference_session, reference_session['trials'])
        data = reference_session[:, reference_session['trials'] == forward_trials[0]]
        for trial in forward_trials[1:]:
            subdata = reference_session[:, reference_session['trials'] == trial]
            data = xr.concat([data, subdata], dim='frame')
    else:
        data = reference_session
    
    pf_reference = pc.PlaceFields(x=data['x'].values,
                                  y=data['y'].values,
                                  t=data['behav_t'].values,
                                  neural_data=data.values,
                                  circular=True,
                                  shuffle_test=True,
                                  nbins=nbins)
    data = data.assign_coords(place_cell_r = ('unit_id', pf_reference.data['spatial_info_pvals'] < alpha))

    for sess in dataset.keys():
        d = dataset[sess] ## data for that session

        if forward:
            forward, reverse = ctb.forward_reverse_trials(d, d['trials'])
            output = d[:, d['trials'] == forward[0]]
            for ftrial in forward:
                sd = d[:, d['trials'] == trial]
                output = xr.concat([output, sd], dim='frame')
        else:
            output = d

        center = ctb.find_center(output['x'].values, output['y'].values)
        for rotation in tqdm(np.arange(0, (360 + degree_increment), degree_increment)):
            rotated_points = np.empty(shape=(2, output['x'].shape[0]))

            for idx, (x, y) in enumerate(zip(output['x'].values, output['y'].values)):
                p = (x, y)
                rot = ctb.rotate(p, origin=center, degrees=rotation)
                rotated_points[0, idx] = rot[0] ## x values
                rotated_points[1, idx] = rot[1] ## y values

            pf_rotated = pc.PlaceFields(x=rotated_points[0],
                                        y=rotated_points[1],
                                        t=output['behav_t'].values,
                                        neural_data=output.values,
                                        circular=True,
                                        shuffle_test=True,
                                        nbins=nbins)
            output = output.assign_coords(place_cell_r = ('unit_id', pf_rotated.data['spatial_info_pvals'] < alpha))

            if neuron_by_neuron:
                if place_cells:
                    neuron_list = np.arange(0, output['unit_id'][(data['place_cell_r']) & (output['place_cell_r'])].shape[0])
                    ref_data = pf_reference.data['normalized_placefields'][(data['place_cell_r']) & (output['place_cell_r'])]
                    rot_data = pf_rotated.data['normalized_placefields'][(data['place_cell_r']) & (output['place_cell_r'])]
                else:
                    neuron_list = np.arange(0, output['unit_id'].shape[0])
                    ref_data = pf_reference.data['normalized_placefields']
                    rot_data = pf_rotated.data['normalized_placefields']

                for neuron in neuron_list:
                    result = pearsonr(ref_data[neuron], rot_data[neuron])
                    result_dict['mouse'].append(mouse)
                    result_dict['session'].append(sess)
                    result_dict['rotation'].append(rotation)
                    result_dict['r'].append(result[0])
                    result_dict['p'].append(result[1])
            else:
                if place_cells:
                    ref_data = pf_reference.data['normalized_placefields'][(data['place_cell_r']) & (output['place_cell_r'])]
                    rot_data = pf_rotated.data['normalized_placefields'][(data['place_cell_r']) & (output['place_cell_r'])]
                else:
                    ref_data = pf_reference.data['normalized_placefields']
                    rot_data = pf_rotated.data['normalized_placefields']

                for bin in np.arange(0, ref_data.shape[1]):
                    result = pearsonr(ref_data.T[bin], rot_data.T[bin]) ## population vector of spatial bin
                    result_dict['mouse'].append(mouse)
                    result_dict['session'].append(sess)
                    result_dict['rotation'].append(rotation)
                    result_dict['r'].append(result[0])
                    result_dict['p'].append(result[1])

## Create dataframe to save
rotation_df = pd.DataFrame(result_dict)
# rotation_df.to_feather(pjoin(save_path, feather_name))

/media/caishuman/csstorage3/Austin/CircleTrack/CircleTrackAnalysis/MultiCon_AfterHours/analysis/../../place_cells.py:44: RuntimeWarning:

invalid value encountered in divide



Doing shuffle tests. This may take a while.


/media/caishuman/csstorage3/Austin/CircleTrack/CircleTrackAnalysis/MultiCon_AfterHours/analysis/../../place_cells.py:458: RuntimeWarning: invalid value encountered in scalar divide
  SI_z = (self.data["spatial_information"][neuron] - np.mean(shuffled_SIs)) / np.std(
/media/caishuman/csstorage3/Austin/CircleTrack/CircleTrackAnalysis/MultiCon_AfterHours/analysis/../../place_cells.py:458: RuntimeWarning: invalid value encountered in scalar divide
  SI_z = (self.data["spatial_information"][neuron] - np.mean(shuffled_SIs)) / np.std(
/media/caishuman/csstorage3/Austin/CircleTrack/CircleTrackAnalysis/MultiCon_AfterHours/analysis/../../place_cells.py:458: RuntimeWarning: invalid value encountered in scalar divide
  SI_z = (self.data["spatial_information"][neuron] - np.mean(shuffled_SIs)) / np.std(
/media/caishuman/csstorage3/Austin/CircleTrack/CircleTrackAnalysis/MultiCon_AfterHours/analysis/../../place_cells.py:458: RuntimeWarning: invalid value encountered in scalar divide
  SI_z = (self.dat

Doing shuffle tests. This may take a while.


/home/austinbaggetta/.conda/envs/calcium/lib/python3.10/site-packages/scipy/stats/_stats_py.py:4424: ConstantInputWarning:

An input array is constant; the correlation coefficient is not defined.

 33%|███▎      | 1/3 [00:46<01:33, 46.83s/it]/media/caishuman/csstorage3/Austin/CircleTrack/CircleTrackAnalysis/MultiCon_AfterHours/analysis/../../place_cells.py:44: RuntimeWarning:

invalid value encountered in divide



...

/home/austinbaggetta/.conda/envs/calcium/lib/python3.10/site-packages/scipy/stats/_stats_py.py:4424: ConstantInputWarning:

An input array is constant; the correlation coefficient is not defined.

 33%|███▎      | 1/3 [00:41<01:23, 41.53s/it]/media/caishuman/csstorage3/Austin/CircleTrack/CircleTrackAnalysis/MultiCon_AfterHours/analysis/../../place_cells.py:44: RuntimeWarning:

invalid value encountered in divide



Doing shuffle tests. This may take a while.


/home/austinbaggetta/.conda/envs/calcium/lib/python3.10/site-packages/scipy/stats/_stats_py.py:4424: ConstantInputWarning:

An input array is constant; the correlation coefficient is not defined.

 67%|██████▋   | 2/3 [01:23<00:41, 41.60s/it]/media/caishuman/csstorage3/Austin/CircleTrack/CircleTrackAnalysis/MultiCon_AfterHours/analysis/../../place_cells.py:44: RuntimeWarning:

invalid value encountered in divide



Doing shuffle tests. This may take a while.


/home/austinbaggetta/.conda/envs/calcium/lib/python3.10/site-packages/scipy/stats/_stats_py.py:4424: ConstantInputWarning:

An input array is constant; the correlation coefficient is not defined.

100%|██████████| 3/3 [02:04<00:00, 41.46s/it]


In [27]:
test = rotation_df.groupby(['mouse', 'session', 'rotation'], as_index=False).agg({'r': ['mean', 'sem']})
test_df = pd.concat([test['mouse'], test['session'], test['rotation'], test['r']], axis=1)
fig_data = test_df
specs = [[{'type': 'polar'}, {'type': 'polar'}]]
fig = pf.custom_graph_template(x_title='', y_title='', font_size=11, rows=1, columns=2, 
                               width=1600, height=400, specs=specs)
for idx, session in enumerate(np.unique(fig_data['session'])):
        if idx == 0:
            fig.add_trace(go.Barpolar(
                r=fig_data['mean'][fig_data['session'] == session],
                theta=fig_data['rotation'][fig_data['session'] == session],
                marker_color='turquoise',
                marker_line_color='black',
                marker_line_width=1,
                opacity=0.8,
                showlegend=False
            ), row=1, col=idx + 1)  
        else:
            fig.add_trace(go.Barpolar(
                r=fig_data['mean'][fig_data['session'] == session],
                theta=fig_data['rotation'][fig_data['session'] == session],
                marker_color='turquoise',
                marker_line_color='black',
                marker_line_width=1,
                opacity=0.8,
                showlegend=False
            ), row=1, col=idx + 1)
    
fig.update_polars(
        radialaxis = dict(showticklabels=True, ticks='', angle=180, tickangle=-180, range=[-0.25, 1], showgrid=True, gridcolor='darkgrey'),
        angularaxis = dict(showticklabels=True, ticks='', showgrid=True, gridcolor='darkgrey')
    )
fig.show()